In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import json
import random

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt

## 1. 读取数据

读取数据并转化为单字符列表。

In [ ]:
with open('./quiz-w10-code/QuanSongCi.txt', 'rb') as f:
    vocabulary = list(f.read().decode('utf-8'))
print('Data size', len(vocabulary))

## 2. 构建单字符的索引

- 统计词频，取最常出现的4999个字符，余下的词统一归为UNK，共5000个字符。
- 构建由字符映射到索引号的字典（dictionary），以及由索引号映射到字符的字典（reverse dictionary）。
- 将原文的字符列表转化为用字符索引表示的数据。

In [ ]:
vocabulary_size = 5000

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))    # 统计词频，取最常出现的4999个字符
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)   # 由字符映射到索引号的字典
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)     # 将原字符列表转化为索引列表
    count[0][1] = unk_count    # UNK的词频
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))     # 由索引号映射到字符的字典
    return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)

## 3. 构建训练数据

依据 Skip-gram 模型，即根据目标词汇预测上下文的机制，构建训练数据：

- 根据所需的上下文长度（skip_window）等距离截取一小段文本，文本长度为 2 * skip_window + 1。
- 每个截取文本中间位置的词作为一条输入数据。
- 每段截取文本中，除了train_data之外的部分，随机取 num_skips 个词作为 label，与输入数据组成 num_skips 条训练数据。
- 将截取文本依次向后移动一个字符重复上述过程，直到组成一个训练 batch。

In [ ]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    abels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 
    buffer = collections.deque(maxlen=span)    # 用于存放输入词和上下文的队列
    if data_index + span > len(data):       # 超过数据长度则从头开始
        data_index = 0
    buffer.extend(data[data_index:data_index + span])     # 将输入词和指定长度的上下文放入队列
    data_index += span
    for i in range(batch_size // num_skips):      # 每个batch里不重复的词数
        context_words = [w for w in range(span) if w != skip_window]    # 上下文出现的词（去掉了该词本身，即span中间的词）
        words_to_use = random.sample(context_words, num_skips)     # 随机选择num_skips个用作label的词
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]     # 输入数据，即span中间的词
            labels[i * num_skips + j, 0] = buffer[context_word]    # label，即从上下文中选出的词
        if data_index == len(data):
            buffer.extend(data[0:span])     # 如果到了文本结尾，则下一条输入数据从文本开头重新开始定义
            data_index = span     # 标记结束位置
        else:
            buffer.append(data[data_index])    # 如果没到文本结尾，则下一条输入数据向后移动一个词
            data_index += 1      # 标记结束位置
    data_index = (data_index + len(data) - span) % len(data)     # 将结束位置向前移动，避免下一个batch时跳过结尾的词
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)

## 4. 构建训练网络

- 初始化 embedding，作为 embeddings 容器，每个值都在-1到1之间随机分布。
- 通过 tf.nn.embedding_lookup 查表操作，获得对应索引的 embedding 结果。
- 初始化 NCE 权重矩阵，用于计算 nce loss。采用 Xavier 初始化，但在距均值两个标准差处进行了截断。
- 利用 NCE 权重矩阵、embedding 后的词向量以及数据的真实 label 计算平均 NCE 损失。
- 采用最基本的梯度下降算法优化器 GradientDescentOptimizer进行优化。
- 用 embedding 矩阵对验证数据进行 embedding 操作，并计算生成的词向量与其他词向量的余弦相似度。

In [ ]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.
num_sampled = 64      # Number of negative examples to sample.

valid_size = 16       # Random set of words to evaluate similarity on.
valid_window = 100    # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()
with graph.as_default():
    
    # 输入数据
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    with tf.device('/cpu:0'):
        # 初始化embedding矩阵
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # 初始化NCE权重矩阵
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    # 计算平均损失
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))
    
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    
    # 计算 embedding 后的验证数据与其他字符的余弦相似度
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    init = tf.global_variables_initializer()

## 5. 进行训练

- 共训练 400000 个 step。
- 每 2000 次输出这两千次的平均损失。
- 每 10000 次计算验证词与其他词的相似度，并输出与验证集中的字符最接近的8个字符。

In [ ]:
num_steps = 400000

with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')
    
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        
    if step % 2000 == 0:
        if step > 0:
            average_loss /= 2000    # 每2000次计算一次平均损失
        print('Average loss at step ', step, ': ', average_loss)
        average_loss = 0

    if step % 10000 == 0:
        sim = similarity.eval()   # 每10000次计算一次相似度
        for i in xrange(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]    # 取8个最相似的字符（去掉自身）
            log_str = 'Nearest to %s:' % valid_word
            for k in xrange(top_k):
                close_word = reverse_dictionary[nearest[k]]    # 用reverse_dictionary将索引号映射回字符
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)
        
    final_embeddings = normalized_embeddings.eval()

# 保存最终生成的embedding
np.save('embedding.npy', final_embeddings)

## 6. 结果可视化

- 用 TSNE 将 embedding 后的结果将为两维，便于可视化呈现。
- 用 matplotlib 绘制结果，呈现词汇的接近程度。

In [ ]:
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18)) 
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.savefig(filename)

mpl.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
mpl.rcParams['axes.unicode_minus'] = False   # 用来正常显示负号

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')    # TSNE降维
plot_only = 500  # 只显示前500个字符
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]   # 用reverse_dictionary将索引号映射回字符
plot_with_labels(low_dim_embs, labels, 'tsne.png')